In [5]:
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder

In [15]:
path = "./data/"
book = pd.read_csv(path+"books.csv")
users = pd.read_csv(path+"users.csv")
train_ratings = pd.read_csv(path+"train_ratings.csv")
test_ratings = pd.read_csv(path+"test_ratings.csv")
data = pd.merge(pd.merge(book, train_ratings, on="isbn"), users, on="user_id")
test_data = pd.merge(pd.merge(book, test_ratings, on="isbn"), users, on="user_id")

data["location_city"] = data["location"].apply(lambda x : x.split(',')[0])
data["location_state"] = data["location"].apply(lambda x : x.split(',')[1])
data["location_country"] =  data["location"].apply(lambda x : x.split(',')[2])

test_data["location_city"] = test_data["location"].apply(lambda x : x.split(',')[0])
test_data["location_state"] = test_data["location"].apply(lambda x : x.split(',')[1])
test_data["location_country"] =  test_data["location"].apply(lambda x : x.split(',')[2])

column = 'category'
data.loc[~data[column].isna(), column] = data.loc[~data[column].isna(), column].apply(lambda x : re.sub("[\'\[\]]","",x).lower())
test_data.loc[~test_data[column].isna(), column] = test_data.loc[~test_data[column].isna(), column].apply(lambda x : re.sub("[\'\[\]]","",x).lower())
# data[column].describe().to_frame()

columns = ["book_title", "year_of_publication", "publisher", "language", "category", "user_id", "age", "location_city", "location_state", "location_country"]
fill_nan = data[columns]
fill_nan2 = test_data[columns]
train_test_ = pd.concat([fill_nan, fill_nan2], axis=0)
for c in fill_nan.columns:
    le = LabelEncoder()
    le.fit(train_test_[c])
    fill_nan[c] = le.transform(fill_nan[c])
    fill_nan2[c] = le.transform(fill_nan2[c])
train_test_ = pd.concat([fill_nan, fill_nan2], axis=0)

/tmp/ipykernel_7820/4092056944.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fill_nan[c] = le.transform(fill_nan[c])
/tmp/ipykernel_7820/4092056944.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fill_nan2[c] = le.transform(fill_nan2[c])


In [ ]:
# pip install deepimpute

In [24]:
train_test_ = train_test_.reset_index(drop=True)

In [26]:
from math import ceil
from deepimpute.multinet import MultiNet
NN_params = {
        # 'loss': "CategoricalCrossentropy",
        'learning_rate': 1e-3,
        'batch_size': 128,
        'max_epochs': 10,
        # 'ncores': 5,
        # 'sub_outputdim': 512,
        # 'architecture': [
            # {"type": "dense", "activation": "relu", "neurons": 200},
            # {"type": "dropout", "activation": "dropout", "rate": 0.3},]
    }
model = MultiNet(**NN_params)
model.fit(train_test_.iloc[:,:-1])

Using all the cores (8)
Input dataset is 383494 cells (rows) and 9 genes (columns)
First 3 rows and columns:
   book_title  year_of_publication  publisher
0       20344                   89       4598
1      131187                   90       9212
2      111089                   92       4937
512 genes selected for imputation
Net 0: 0 predictors, 512 targets
Normalization
Building network
[{'type': 'dense', 'neurons': 256, 'activation': 'relu'}, {'type': 'dropout', 'rate': 0.2}]


/opt/conda/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Fitting with 383494 cells
Epoch 1/10
2847/2847 [==============================] - 11s 4ms/step - loss: 340.2342 - val_loss: 231.7168
Epoch 2/10
2847/2847 [==============================] - 10s 4ms/step - loss: 152.1802 - val_loss: 88.5197
Epoch 3/10
2847/2847 [==============================] - 11s 4ms/step - loss: 50.9083 - val_loss: 23.7033
Epoch 4/10
2847/2847 [==============================] - 11s 4ms/step - loss: 12.2770 - val_loss: 5.8607
Epoch 5/10
2847/2847 [==============================] - 11s 4ms/step - loss: 4.6901 - val_loss: 4.3592
Epoch 6/10
2847/2847 [==============================] - 11s 4ms/step - loss: 4.3490 - val_loss: 4.3580
Epoch 7/10
2847/2847 [==============================] - 11s 4ms/step - loss: 4.3489 - val_loss: 4.3579
Epoch 8/10
2847/2847 [==============================] - 11s 4ms/step - loss: 4.3489 - val_loss: 4.3578
Epoch 9/10
2847/2847 [==============================] - 11s 4ms/step - loss: 4.3489 - val_loss: 4.3580
Epoch 10/10
2847/2847 [==============

In [14]:
fill_nan = model.predict(fill_nan)
fill_nan2 = model.predict(fill_nan2)

2397/2397 [==============================] - 4s 2ms/step
Filling zeros


,book_title,year_of_publication,publisher,language,category,user_id,age,location_city,location_state,location_country
0,20344.0,89.0,4598.0,4.0,144.0,2719.0,91.0,7297.0,913.0,193.0
1,76544.0,81.0,820.0,26.0,4259.0,2719.0,91.0,7297.0,913.0,193.0
2,12356.0,82.0,8073.0,4.0,2398.0,2719.0,91.0,7297.0,913.0,193.0
3,75957.0,87.0,10204.0,4.0,2842.0,2719.0,91.0,7297.0,913.0,193.0
4,87161.0,90.0,983.0,4.0,2759.0,2719.0,91.0,7297.0,913.0,193.0
...,...,...,...,...,...,...,...,...,...,...
76694,28537.0,82.0,11189.0,26.0,4259.0,67697.0,91.0,7203.0,913.0,111.0
76695,117178.0,72.0,7854.0,26.0,4259.0,67716.0,91.0,11645.0,913.0,206.0
76696,59089.0,78.0,5740.0,8.0,4259.0,67745.0,30.0,4166.0,71.0,105.0
76697,95831.0,82.0,1065.0,4.0,3932.0,67921.0,36.0,7487.0,784.0,322.0


In [28]:
fill_nan

,book_title,year_of_publication,publisher,language,category,user_id,age,location_city,location_state,location_country
0,20344,89,4598,4,144,0,91,10869,1055,56
1,131187,90,9212,4,2398,0,91,10869,1055,56
2,111089,92,4937,26,4259,0,91,10869,1055,56
3,53653,87,6770,26,4259,0,91,10869,1055,56
4,120495,86,1595,26,4259,0,91,10869,1055,56
...,...,...,...,...,...,...,...,...,...,...
306790,96959,88,10985,26,4259,67957,49,10483,1381,322
306791,42161,87,9552,4,689,68012,69,11306,397,56
306792,116946,90,8549,26,4259,68014,91,5059,50,322
306793,124396,69,983,26,4259,68022,28,11227,1511,322


In [31]:
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

# reg = CatBoostRegressor(iterations=10000, learning_rate=1e-1, random_seed=777, task_type="GPU")
# reg.fit(new_train.iloc[:,:-1], new_train.iloc[:, -1])
# pred = reg.predict(new_test)

xgb_model = XGBRegressor(n_estimators=1000, learning_rate=0.1, max_depth=10, verbose=1, random_state=123)
xgb_model.fit(fill_nan, data['rating'])
pred = xgb_model.predict(fill_nan2)

[08:12:09] WARNING: ../src/learner.cc:627: 
Parameters: { "verbose" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




In [32]:
submit = pd.read_csv("/opt/ml/input/code/jmkim/level1_bookratingprediction_recsys-level-recsys-07/data/sample_submission.csv")
submit.rating = pred
submit.to_csv("MultiNet-XGBREG.csv",index=False)

In [ ]:
# import pandas as pd
# import numpy as np
# import os
# import re
# import matplotlib.pyplot as plt
# # import seaborn as sns
# import scipy.stats as spst
# import sklearn
# from catboost import CatBoostClassifier
# from sklearn.impute import KNNImputer
# from sklearn.impute import IterativeImputer
# from sklearn.preprocessing import LabelEncoder
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.preprocessing import OneHotEncoder

# path = "./data/"
# book = pd.read_csv(path+"books.csv")
# users = pd.read_csv(path+"users.csv")
# train_ratings = pd.read_csv(path+"train_ratings.csv")
# test_ratings = pd.read_csv(path+"test_ratings.csv")
# data = pd.merge(pd.merge(book, train_ratings, on="isbn"), users, on="user_id")
# test_data = pd.merge(pd.merge(book, test_ratings, on="isbn"), users, on="user_id")

# data["location_city"] = data["location"].apply(lambda x : x.split(',')[0])
# data["location_state"] = data["location"].apply(lambda x : x.split(',')[1])
# data["location_country"] =  data["location"].apply(lambda x : x.split(',')[2])

# test_data["location_city"] = test_data["location"].apply(lambda x : x.split(',')[0])
# test_data["location_state"] = test_data["location"].apply(lambda x : x.split(',')[1])
# test_data["location_country"] =  test_data["location"].apply(lambda x : x.split(',')[2])

# column = 'category'
# data.loc[~data[column].isna(), column] = data.loc[~data[column].isna(), column].apply(lambda x : re.sub("[\'\[\]]","",x).lower())
# test_data.loc[~test_data[column].isna(), column] = test_data.loc[~test_data[column].isna(), column].apply(lambda x : re.sub("[\'\[\]]","",x).lower())
# # data[column].describe().to_frame()

# columns = ["book_title", "year_of_publication", "publisher", "language", "category", "user_id", "age", "location_city", "location_state", "location_country"]
# fill_nan = data[columns]
# fill_nan2 = test_data[columns]
# train_test_ = pd.concat([fill_nan, fill_nan2], axis=0)
# for c in fill_nan.columns:
#     le = LabelEncoder()
#     le.fit(train_test_[c])
#     fill_nan[c] = le.transform(fill_nan[c])
#     fill_nan2[c] = le.transform(fill_nan2[c])
# imp = KNNImputer()
# fill_nan = imp.fit_transform(fill_nan)
# fill_nan2 = imp.transform(fill_nan2)
# # imp = IterativeImputer(
# #     estimator=[(CatBoostClassifier(verbose=False), slice(1, 10))],
# #     transformers=[(OneHotEncoder(sparse=False), slice(1, 10))],
# #     initial_strategy="most_frequent",
# #     verbose=1
# # )
# # X_filled = imp.fit_transform(fill_nan)
# # print(X_filled)